In [ ]:
import tkinter
from tkinter import *
import StrongAdversarial
from StrongAdversarial import *
import WeakAdversarial
from WeakAdversarial import *
from tkinter import ttk
import threading
import matplotlib
import tensorflow as tf
import numpy as np
from PIL import ImageTk, Image
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import decode_predictions, preprocess_input
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from tensorflow.keras.models import model_from_json
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Sequential
import pandas as pd

In [ ]:
def to_image(img):
    img = (img - img.min()) / (img.max() - img.min())
    return img

def decode(labels):
    classes = {
    0: "Speed limit\n (20km/h)",
    1: "Speed limit\n (30km/h)",
    2: "Speed limit\n (50km/h)",
    3: "Speed limit\n (60km/h)",
    4: "Speed limit\n (70km/h)",
    5: "Speed limit\n (80km/h)",
    6: "End of speed\n limit (80km/h)",
    7: "Speed limit\n (100km/h)",
    8: "Speed limit\n (120km/h)",
    9: "No passing",
    10: "No passing for\nheavy vehicles",
    11: "Right-of-way",
    12: "Priority road",
    13: "Yield",
    14: "Stop Sign",
    15: "No vehicles",
    16: "No heavy\nvehicles",
    17: "Do Not Enter",
    18: "General\ncaution",
    19: "Dangerous\nleft curve",
    20: "Dangerous\nright curve",
    21: "Double curve",
    22: "Bumpy road",
    23: "Slippery road",
    24: "Right narrows",
    25: "Road work",
    26: "Traffic\nsignals",
    27: "Pedestrians",
    28: "Children\ncrossing",
    29: "Bicycles\ncrossing",
    30: "Beware of\nice/snow",
    31: "Wild animals\ncrossing",
    32: "End speed\nlimits",
    33: "Turn right\nahead",
    34: "Turn left\nahead",
    35: "Ahead only",
    36: "Go straight\nor right",
    37: "Go straight\nor left",
    38: "Keep right",
    39: "Keep left",
    40: "Roundabout\nmandatory",
    41: "End of no\npassing",
    42: "End of no\npassing by\nheavy vehicles",
  }
    new = []
    for i in labels:
        new.append(classes[i])
    return new


def load_and_format_image(path, size):
    image = load_img(path, target_size=size)
    image = img_to_array(image)/255
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = tf.convert_to_tensor(image)
    return image

class MainGUI:
    def __init__(self, net, size=(224, 224), default_path='grey.jpg'):
        self.window = tkinter.Tk()
        self.window.title('Image Predictor')
        self.window.configure(background='#fff')
        self.window.resizable(0, 0)
        self.window.geometry('684x624')
        # self.window.geometry('780x624')
        self.path = default_path
        self.size = size
        self.net = net
        self.input_image = tf.ones(shape=(1, self.size[0], self.size[1], 3)) * 0.5
        self.current_image = None
        self.adv_image = None
        self.diff = None
        self.canvas = None
        self.button_frame = tkinter.Frame(self.window, bg='#fff')
        self.choose_img_btn = tkinter.Button(self.button_frame, text='Choose image',
                                             command=self.click_choose_image_btn)
        self.create_adversarial_btn = tkinter.Button(self.button_frame, text='Choose adversarial',
                                                     command=self.click_adversarial_btn)
        self.choose_img_btn.grid(row=0, column=0, sticky='we', padx=(0, 200))
        self.create_adversarial_btn.grid(row=0, column=1, sticky='we', padx=(200, 0))
        self.button_frame.columnconfigure(0, weight=1)
        self.button_frame.columnconfigure(1, weight=1)
        self.button_frame.pack(fill='x')
        self.image_frame = tkinter.Frame(self.window, bg='#fff')
        default_img = Image.open(self.path)
        default_img = default_img.resize(self.size, Image.ANTIALIAS)
        img_tk = ImageTk.PhotoImage(default_img)
        self.first_img_label = tkinter.Label(self.image_frame, text='Initial Image', bg='#fff')
        self.first_img_label.grid(row=2, column=0)
        self.first_img = tkinter.Label(self.image_frame, image=img_tk, bg='#fff')
        self.first_img.grid(row=1, column=0)
        self.second_img_label = tkinter.Label(self.image_frame, text='Adversarial Example', bg='#fff')
        self.second_img_label.grid(row=2, column=1)
        self.second_img = tkinter.Label(self.image_frame, image=img_tk, bg='#fff')
        self.second_img.grid(row=1, column=1)
        self.third_img_label = tkinter.Label(self.image_frame, text='Normalized Diff.', bg='#fff').grid(row=2, column=2)
        self.third_img = tkinter.Label(self.image_frame, image=img_tk, bg='#fff')
        self.third_img.grid(row=1, column=2)
        label = tkinter.Label(self.image_frame, text='Click on an image to see top 5 predictions', bg='#fff')
        label.grid(row=3, column=0, columnspan=3, sticky='nswe')
        self.image_frame.pack(fill='both')
        self.window.bind('<Control-s>', lambda e: self.save_adv_example())
        self.window.mainloop()

    def click_choose_image_btn(self):
        self.path = tkinter.filedialog.askopenfilename(initialdir='./files', title='Select Image',
                                                       filetypes=(("png files", "*.png"), ("jpeg files", "*.jpg")))
        if self.path is None or len(self.path) == 0:
            return
        self.current_image = self.path
        print(self.current_image)
        new_img = Image.open(self.path)
        new_img = new_img.resize((self.size[0], self.size[1]), Image.ANTIALIAS)
        new_img_tk = ImageTk.PhotoImage(new_img)
        self.first_img.configure(image=new_img_tk)
        
        default_img = Image.open("grey.jpg")
        default_img = default_img.resize((224,224), Image.ANTIALIAS)
        img_tk = ImageTk.PhotoImage(default_img)
        self.second_img.configure(image=img_tk)
        self.second_img.image = img_tk
        self.third_img.configure(image=img_tk)
        self.third_img.image = img_tk
        self.first_img.image = new_img_tk
        self.input_image = load_and_format_image(self.path, self.size)
        self.show_prediction2(self.input_image, self.first_img)

    def display_barh(self, labels, probabilities):
        plt.close('all')
        fig, ax = plt.subplots(figsize=(8, 4.5), dpi=75)

        y_pos = np.arange(len(labels))

        ax.barh(y_pos, probabilities, align='center', color='blue')
        ax.set_yticks(y_pos)

        ax.set_yticklabels(labels)
        ax.invert_yaxis()  # labels read top-to-bottom
        ax.set_xlabel('Probability\n')
        ax.set_title('Estimation of model')

        if self.canvas is not None:
            self.canvas.get_tk_widget().pack_forget()
        self.canvas = FigureCanvasTkAgg(fig, master=self.image_frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().grid(row=3, column=0, columnspan=3, sticky='nswe')

    def labels_and_probabilities(self, image):
        predictions = tf.nn.softmax(self.net(image))
        predictions = tf.nn.top_k(predictions, k=5, sorted=True, name=None)
        labels = predictions.indices[0].numpy()
        labels = decode(labels)
        probabilities = predictions.values[0].numpy()

        return labels, probabilities

    def show_prediction(self, image, selected_label):
        self.first_img.config(bg='#fff')
        self.second_img.config(bg='#fff')
        selected_label.config(bg='#06f')
        if image is None:
            return
        labels, probabilities = self.labels_and_probabilities(image)
        self.display_barh(labels, probabilities)
        
    def labels_and_probabilities2(self, image):
        predictions = self.net(image)
        predictions = tf.nn.top_k(predictions, k=5, sorted=True, name=None)
        labels = predictions.indices[0].numpy()
        labels = decode(labels)
        probabilities = predictions.values[0].numpy()

        return labels, probabilities

    def show_prediction2(self, image, selected_label):
        self.first_img.config(bg='#fff')
        self.second_img.config(bg='#fff')
        selected_label.config(bg='#06f')
        if image is None:
            return
        labels, probabilities = self.labels_and_probabilities2(image)
        self.display_barh(labels, probabilities)

    def click_adversarial_btn(self):
        default_img = Image.open("grey.jpg")
        default_img = default_img.resize(self.size, Image.ANTIALIAS)
        default_img = ImageTk.PhotoImage(default_img)
        self.second_img.configure(image=default_img)
        self.second_img.image = default_img
        self.third_img.configure(image=default_img)
        self.third_img.image = default_img
        way=[]
        target=[]
        result=[]
        def confirm(window,var,drop):
            way.append(var.get())
            target.append(drop.current())
            window.destroy()
            if way[0]==0:
                with open('./Model/MobileNetV2_Model.json', 'r') as f:
                    model0 = model_from_json(f.read())
                model0.load_weights('./Model/MobileNetV2_Weights.h5')
                x=WeakAdversarial.main(model0,self.current_image,target[0])
                result.append(x)
                plt.imsave("adv_img.png",result[0][0])
                new_img = Image.open("adv_img.png")
                new_img = new_img.resize((self.size[0], self.size[1]), Image.ANTIALIAS)
                new_img_tk = ImageTk.PhotoImage(new_img)
                self.second_img.configure(image=new_img_tk)
                self.second_img.image = new_img_tk
                self.show_prediction2(x, self.second_img)

                dif = (self.input_image - x[0]).numpy()
                dif = np.reshape(dif, (self.size[0], self.size[1], 3))
                dif = to_image(dif)
                dif = dif * 255
                dif = np.uint8(dif)
                new_dif_img_tk = ImageTk.PhotoImage(Image.fromarray(dif))
                self.third_img.configure(image=new_dif_img_tk)
                self.third_img.image = new_dif_img_tk
            if way[0]==1:
                with open('./Model/MobileNetV2_Model.json', 'r') as f:
                    model1 = model_from_json(f.read())
                m = Sequential()
                for l in model.layers[:-1]:
                    m.add(l)
                m.add(tf.keras.layers.Dense(43))
                model1 = m
                model1.load_weights('./Model/MobileNetV2_Weights.h5')
                x = StrongAdversarial.main(model1,self.current_image,target[0])
                plt.imsave("adv_img.png",x[0])

                new_img = Image.open("adv_img.png")
                new_img = new_img.resize((self.size[0], self.size[1]), Image.ANTIALIAS)
                new_img_tk = ImageTk.PhotoImage(new_img)
                self.second_img.configure(image=new_img_tk)
                self.second_img.image = new_img_tk
                self.show_prediction2(x, self.second_img)

                dif = (self.input_image - x[0]).numpy()
                dif = np.reshape(dif, (self.size[0], self.size[1], 3))
                dif = to_image(dif)
                dif = dif * 255
                dif = np.uint8(dif)
                new_dif_img_tk = ImageTk.PhotoImage(Image.fromarray(dif))
                self.third_img.configure(image=new_dif_img_tk)
                self.third_img.image = new_dif_img_tk
            
        t = tkinter.Toplevel(self.window)
        t.resizable(0, 0)
        t.geometry("315x200")
        Label(t,text='Choose Target:').grid(row=0,column=0,padx=(70,0),pady=(10,0))
        v = tkinter.IntVar()
        choose_target = ttk.Combobox(t)
        choose_target['value'] = ("Speed limit (20km/h)",
                                   "Speed limit (30km/h)",
                                   "Speed limit (50km/h)",
                                   "Speed limit (60km/h)",
                                   "Speed limit (70km/h)",
                                   "Speed limit (80km/h)",
                                   "End of speed limit (80km/h)",
                                   "Speed limit (100km/h)",
                                   "Speed limit (120km/h)",
                                   "No passing",
                                   "No passing for heavy vehicles",
                                   "Right-of-way",
                                   "Priority road",
                                   "Yield",
                                   "Stop Sign",
                                   "No vehicles",
                                   "No heavy vehicles",
                                   "Do Not Enter",
                                   "General caution",
                                   "Dangerous left curve",
                                   "Dangerous right curve",
                                   "Double curve",
                                   "Bumpy road",
                                   "Slippery road",
                                   "Right narrows",
                                   "Road work",
                                   "Traffic signals",
                                   "Pedestrians",
                                   "Children crossing",
                                   "Bicycles crossing",
                                   "Beware of ice/snow",
                                   "Wild animals crossing",
                                   "End speed limits",
                                   "Turn right ahead",
                                   "Turn left ahead",
                                   "Ahead only",
                                   "Go straight or right",
                                   "Go straight or left",
                                   "Keep right",
                                   "Keep left",
                                   "Roundabout mandatory",
                                   "End of no passing",
                                   "End of no passing by heavy vehicles")
        choose_target.grid(row=1,column=0,padx=(70,0),pady=(10,0))
        choose_target.current(0)
        Label(t,text='Choose Adversarial Attack:').grid(row=2,column=0,padx=(70,0),pady=(10,0))
        tkinter.Radiobutton(t, text="Weak(Fast)", variable=v, value=0).grid(row=3,column=0,padx=(75,0))
        tkinter.Radiobutton(t, text="Strong(Slow)", variable=v, value=1).grid(row=4,column=0,padx=(80,0))
        Button(t,text="Confirm",command = lambda:confirm(t,v,choose_target)).grid(row=5,column=0,padx=(80,0),pady=(10,0))



In [ ]:
with open('./Model/MobileNetV2_Model.json', 'r') as f:
    model = model_from_json(f.read())
model.load_weights('./Model/MobileNetV2_Weights.h5')

In [ ]:
MainGUI(model)